In [26]:
# Istalando pacotes necessários
%pip install geopandas matplotlib contextily shapely


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import box, Point, Polygon, MultiPolygon
from shapely.wkt import loads

In [28]:
# lendo o arquivo .shp
caminho_shp = '../databases/paradas/paradas.shp'
df = gpd.read_file(caminho_shp)

In [29]:
# pegando algumas informações
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5457 entries, 0 to 5456
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   parada      5457 non-null   object  
 1   descricao   5456 non-null   object  
 2   situacao    5457 non-null   object  
 3   estrutura_  4948 non-null   object  
 4   tipo        5455 non-null   object  
 5   geometry    5457 non-null   geometry
dtypes: geometry(1), object(5)
memory usage: 255.9+ KB


In [30]:
# O que temos na coluna 'geometry' ?
df['geometry']

0       POINT (203628.051 8238910.255)
1       POINT (203626.569 8238879.548)
2       POINT (203515.071 8238613.416)
3       POINT (203481.047 8238551.784)
4       POINT (203579.703 8239213.734)
                     ...              
5452    POINT (167354.394 8264495.757)
5453    POINT (167311.355 8264500.454)
5454    POINT (163265.979 8265332.396)
5455    POINT (162912.871 8269061.030)
5456    POINT (163217.911 8264159.565)
Name: geometry, Length: 5457, dtype: geometry

In [31]:
# Qual sistemas estamos?
# acessando o atributo crs
df.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [32]:
# O folium trabalha no sistema WGS 84(EPSG:4326), o correto é converter para esse formato
df = df.to_crs('EPSG:4326')

In [33]:
# Agora conseguimos plotar

# Pegando uma cordenada pelo indice
x = df.geometry.x.iloc[0] # longitude
y = df.geometry.y.iloc[0] # latitute
print(f'Coordenada x é longitude e vale: {x}\nCoordena y é latitude e vale: {y}')

Coordenada x é longitude e vale: -47.76791017881488
Coordena y é latitude e vale: -15.911319542624627


In [34]:
import folium

# Criar um mapa Folium centrado nas coordenadas do ponto selecionado
mapa = folium.Map(location=[y, x], zoom_start=12)

# Adicionar um marcador para o ponto no mapa
folium.Marker(location=[y, x]).add_to(mapa)

# Exibir o mapa
mapa

In [35]:
# verificando se faz sentido o plot
df['descricao'].iloc[0]

'Avenida Morro da Cruz'

In [36]:
# Lendo o arquivo das linhas
file = '../databases/linhas/Linhas.shp'
linhas = gpd.read_file(file)
linhas.head(5)
linhas.linha.shape[0]

1308

In [37]:
linestringRecanto = linhas[linhas['linha']=='813.2']
linestringRecantoIda = linestringRecanto.iloc[1] # ida
linestringRecantoVolta = linestringRecanto.iloc[0] # volta


In [38]:
import folium
from shapely.geometry import LineString

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Criar um mapa inicial
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring1 = linestringRecanto.geometry.iloc[1] # Ida
linestring2 = linestringRecanto.geometry.iloc[0] # Volta

# Converter as coordenadas do Linestring para o formato que o Folium espera (latitude, longitude)
lat_lon_coordsIda = [(lat1, lon1) for lon1, lat1 in linestring1.coords]
lat_lon_coordsVolta = [(lat2, lon2) for lon2, lat2 in linestring2.coords]


# Adicionar o Linestring ao mapa como uma polilinha (linha poligonal)

# Plotando a ida
folium.PolyLine(locations=lat_lon_coordsIda, color='blue', tooltip=linestringRecanto['linha'].iloc[1]+ ' ' + linestringRecanto['sentido'].iloc[1]).add_to(m)

# Plotando a volta
folium.PolyLine(locations=lat_lon_coordsVolta, color='red', tooltip=linestringRecanto['linha'].iloc[0]+ ' ' + linestringRecanto['sentido'].iloc[0]).add_to(m)

# Mostrar o mapa
m.save('mapaLinestring.html')
m

In [39]:
# Visualizando os rótulos  do dataframe das paradas
paradas = gpd.read_file("../databases/paradas/paradas.shp")
paradas.tail(5)

,parada,descricao,situacao,estrutura_,tipo,geometry
5452,1163,NUCLEO RURAL RODEADOR - LADO OPOSTO ESCOLA ROD...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (167354.394 8264495.757)
5453,1164,NUCLEO RURAL RODEADOR - EM FRENTE ESCOLA RODEA...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (167311.355 8264500.454)
5454,1166,ANTES DA ENTRADA PARA A DF 445 NORTE - KM 7 DF...,ATIVA,SEM ESTRUTURA,Tipo C,POINT (163265.979 8265332.396)
5455,1159,LADO OPOSTO A ENTRADA DA RADIOBRAS E DF 220 DF...,ATIVA,SEM ESTRUTURA,Habitual,POINT (162912.871 8269061.030)
5456,6374,Km 07 - Gleba 02 - Chácara 116 - Chácara Beatr...,ATIVA,SEM ESTRUTURA,Padrão,POINT (163217.911 8264159.565)


In [40]:
# convertendo para o sistema adequado
paradas = paradas.to_crs('EPSG:4326')
paradas

,parada,descricao,situacao,estrutura_,tipo,geometry
0,6710,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76791 -15.91132)
1,6711,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76793 -15.91160)
2,6712,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76900 -15.91399)
3,6713,Avenida Morro da Cruz,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76933 -15.91454)
4,6827,Residencial Vitoria,DESATIVADA,SEM ESTRUTURA,Habitual,POINT (-47.76832 -15.90857)
...,...,...,...,...,...,...
5452,1163,NUCLEO RURAL RODEADOR - LADO OPOSTO ESCOLA ROD...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.10282 -15.67575)
5453,1164,NUCLEO RURAL RODEADOR - EM FRENTE ESCOLA RODEA...,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.10322 -15.67570)
5454,1166,ANTES DA ENTRADA PARA A DF 445 NORTE - KM 7 DF...,ATIVA,SEM ESTRUTURA,Tipo C,POINT (-48.14080 -15.66765)
5455,1159,LADO OPOSTO A ENTRADA DA RADIOBRAS E DF 220 DF...,ATIVA,SEM ESTRUTURA,Habitual,POINT (-48.14358 -15.63395)


In [41]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring = linestringRecanto.geometry.iloc[0]

# Lista para armazenar as paradas encontradas
paradas_encontradas = []

# Conjunto para armazenar os números das paradas encontradas
numeros_paradas_encontradas = set()

# Função para aproximar as coordenadas para 2 casas decimais
def aproximar_coordenadas(coord):
    return round(coord, 2)

# Iterar sobre os pontos de paradas e verificar se estão contidos na Linestring
for idx, ponto_parada in paradas.iterrows():
    numero_parada = ponto_parada["parada"]
    ponto = ponto_parada["geometry"]
    ponto_aproximado = Point(aproximar_coordenadas(ponto.x), aproximar_coordenadas(ponto.y))
    linestring_aproximada = LineString([Point(aproximar_coordenadas(x), aproximar_coordenadas(y)) for x, y in linestring.coords])
    if linestring_aproximada.contains(ponto_aproximado) and numero_parada not in numeros_paradas_encontradas:
        paradas_encontradas.append(ponto_parada)
        numeros_paradas_encontradas.add(numero_parada)

# Criar um novo GeoDataFrame com as paradas encontradas
gdf_paradas_encontradas = gpd.GeoDataFrame(paradas_encontradas, crs='EPSG:4326')
display(gdf_paradas_encontradas)
# Plotar as paradas de ônibus no mapa com o Folium
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

for idx, ponto_parada in gdf_paradas_encontradas.iterrows():
    lat, lon = ponto_parada["geometry"].y, ponto_parada["geometry"].x
    folium.Marker([lat, lon], popup=ponto_parada['descricao']).add_to(m)

# Mostrar o mapa com as paradas de ônibus
m.save('mapa_paradas_onibus.html')


,parada,descricao,situacao,estrutura_,tipo,geometry
69,3517,EQN 513/514 W3 NORTE,ATIVA,ACOSTAMENTO OU BAIA,Tradicional,POINT (-47.89601 -15.74749)
70,3473,SQN 113 EIXO W NORTE,ATIVA,ACOSTAMENTO OU BAIA,Tradicional,POINT (-47.88979 -15.74707)
119,2649,BL. 300 NB 1,ATIVA,SEM ESTRUTURA,Placa,POINT (-47.96304 -15.86636)
120,2672,EM FRENTE AO BL. 211 NB 1,ATIVA,ACOSTAMENTO OU BAIA,Cemusa,POINT (-47.96287 -15.86609)
121,6057,FRENTE A RODA DO CHOPP EPNB,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-47.96680 -15.86686)
...,...,...,...,...,...,...
5292,3291,EQS 106/107 - Cine Brasília Eixo W Sul,ATIVA,ACOSTAMENTO OU BAIA,Cemusa,POINT (-47.89929 -15.81486)
5293,3343,EQS 106/107 - Cine Brasília Eixo W Sul,ATIVA,ACOSTAMENTO OU BAIA,Cemusa,POINT (-47.89912 -15.81473)
5300,3383,EQS 106/107 - CINE BRASILIA EIXO W SUL,ATIVA,ACOSTAMENTO OU BAIA,Fibra de Vidro,POINT (-47.89896 -15.81460)
5353,4723,APOS O VIADUTO DE ACESSO AO LAGO NORTE BR-020,ATIVA,SEM ESTRUTURA,Tipo C,POINT (-47.89611 -15.72536)


In [42]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

# Criar o objeto Linestring
linestringRecanto = linestringRecanto.to_crs('EPSG:4326')

# Obter o objeto LineString a partir da coluna "geometry" do DataFrame
linestring = linestringRecanto.geometry.iloc[0]

# Lista para armazenar as paradas encontradas
paradas_encontradas = []

# Função para aproximar as coordenadas para 2 casas decimais

#def aproximar_coordenadas(coord):
#    return round(coord, )

# Definir o limite de proximidade aceitável (em graus)
limite_proximidade = 0.0002

#Iterar sobre os pontos de paradas e verificar se estão próximos da Linestring
for idx, ponto_parada in paradas.iterrows():
    ponto = ponto_parada["geometry"]
    distancia = linestring.distance(ponto)
    if distancia <= limite_proximidade:
        paradas_encontradas.append(ponto_parada)


# Criar um novo GeoDataFrame com as paradas encontradas
gdf_paradas_encontradas = gpd.GeoDataFrame(paradas_encontradas, crs='EPSG:4326')
display(gdf_paradas_encontradas)
# Plotar as paradas de ônibus no mapa com o Folium
m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

for idx, ponto_parada in gdf_paradas_encontradas.iterrows():
    lat, lon = ponto_parada["geometry"].y, ponto_parada["geometry"].x
    folium.Marker([lat, lon], popup=ponto_parada['parada']).add_to(m)

# Mostrar o mapa com as paradas de ônibus
m.save('mapa_paradas_onibus.html')
m

,parada,descricao,situacao,estrutura_,tipo,geometry
69,3517,EQN 513/514 W3 NORTE,ATIVA,ACOSTAMENTO OU BAIA,Tradicional,POINT (-47.89601 -15.74749)
122,6058,FRENTE AO BLOCO 440 EPNB,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-47.96505 -15.86656)
123,6059,FRENTE AO BLOCO 91 EPNB,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-47.96109 -15.86449)
226,4027,QN 01 CONJ. 16 CASA 13 APOS A PASSARELA DF 075,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.00549 -15.87796)
227,4028,QN 01 CONJ. 31 CASA 24 DF 075,ATIVA,ACOSTAMENTO OU BAIA,Tipo C,POINT (-48.00207 -15.87766)
...,...,...,...,...,...,...
5211,3572,ENTREQUADRA 509/508 - HOSPITAL DIA W3 SUL,ATIVA,ACOSTAMENTO OU BAIA,Tradicional,POINT (-47.90791 -15.81387)
5243,3470,LATERAL HOSPITAL SANTA HELENA LIGACAO,ATIVA,SEM ESTRUTURA,Tipo C,POINT (-47.89732 -15.73542)
5249,3520,EM FRENTE JUSTICA DO TRABALHO LIGACAO,ATIVA,SEM ESTRUTURA,Cemusa,POINT (-47.89838 -15.73702)
5255,3519,EQN 515/516 - BRB W3 NORTE,ATIVA,ACOSTAMENTO OU BAIA,Tradicional,POINT (-47.89805 -15.74110)


In [43]:
linestringLinhas = linhas[linhas['sentido']=='IDA']
for i in range(linestringLinhas.shape[0]):
     line = linestringLinhas.iloc[i]
     display(line.linha)

'0.018'

'0.946'

'0.373'

'503.3'

'531.1'

...

'0.844'

'0.172'

'805.6'

'506.2'

'813.3'

In [44]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium
import pandas as pd

# Filtrando a linestring apenas pelas linhas de onibus de ida
linestringLinhas = linhas[linhas['sentido']=='IDA']
linestringLinhas = linestringLinhas.to_crs('EPSG:4326')
# Criando um pandasdatagrame para armazenar todas as nossas linhas
df = pd.DataFrame(columns=['linha', 'parada'])

# Fazendo um loop para percorrer todas as linhas de onibus
for i in range(linestringLinhas.shape[0]):
    line = linestringLinhas.iloc[i]
    linestring = linestringLinhas.geometry.iloc[i]   

    # Criando listas para armazenar as paradas e dados dos ids da paradas encontradas
    paradas_encontradas = []
    data = []

    limite_proximidade = 0.0002

    # Iterar sobre os pontos de paradas e verificar se estão próximos da Linestring
    for idx, ponto_parada in paradas.iterrows():
        ponto = ponto_parada["geometry"]
        distancia = linestring.distance(ponto)
        if distancia <= limite_proximidade:
            paradas_encontradas.append(ponto_parada)

    # Criar um novo GeoDataFrame com as paradas encontradas
    gdf_paradas_encontradas = gpd.GeoDataFrame(paradas_encontradas, crs='EPSG:4326')

    # Plotar as paradas de ônibus no mapa com o Folium
    # m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

    for idx, ponto_parada in gdf_paradas_encontradas.iterrows():
        lat, lon = ponto_parada["geometry"].y, ponto_parada["geometry"].x
        data.append([lat, lon, ponto_parada['parada']])
        # folium.Marker([lat, lon], popup=ponto_parada['parada']).add_to(m)
    df.loc[i] = [line.linha, data]

/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [45]:
import os

if not os.path.exists('../databases/luiz_dataset'):
    os.makedirs('../databases/luiz_dataset')
df.to_csv('../databases/luiz_dataset/paradas_linestrings_ida.csv')

In [46]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium
import pandas as pd

# Filtrando a linestring apenas pelas linhas de onibus de ida
linestringLinhas = linhas[linhas['sentido']=='VOLTA']
linestringLinhas = linestringLinhas.to_crs('EPSG:4326')
# Criando um pandasdatagrame para armazenar todas as nossas linhas
df = pd.DataFrame(columns=['linha', 'parada'])

# Fazendo um loop para percorrer todas as linhas de onibus
for i in range(linestringLinhas.shape[0]):
    line = linestringLinhas.iloc[i]
    linestring = linestringLinhas.geometry.iloc[i]   

    # Criando listas para armazenar as paradas e dados dos ids da paradas encontradas
    paradas_encontradas = []
    data = []

    limite_proximidade = 0.0002

    # Iterar sobre os pontos de paradas e verificar se estão próximos da Linestring
    for idx, ponto_parada in paradas.iterrows():
        ponto = ponto_parada["geometry"]
        distancia = linestring.distance(ponto)
        if distancia <= limite_proximidade:
            paradas_encontradas.append(ponto_parada)

    # Criar um novo GeoDataFrame com as paradas encontradas
    gdf_paradas_encontradas = gpd.GeoDataFrame(paradas_encontradas, crs='EPSG:4326')

    # Plotar as paradas de ônibus no mapa com o Folium
    # m = folium.Map(location=[-15.76050, -47.78134], zoom_start=10)

    for idx, ponto_parada in gdf_paradas_encontradas.iterrows():
        lat, lon = ponto_parada["geometry"].y, ponto_parada["geometry"].x
        data.append([lat, lon, ponto_parada['parada']])
        # folium.Marker([lat, lon], popup=ponto_parada['parada']).add_to(m)
    df.loc[i] = [line.linha, data]
    

/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [47]:
df

,linha,parada
0,0.018,"[[-15.767293, -47.788723, 7557], [-15.77012499..."
1,100.2,"[[-15.858748568353112, -47.92771717852002, 250..."
2,531.1,"[[-15.638937404577504, -47.82260762728512, 491..."
3,396.2,"[[-15.866860500781998, -47.96679853983349, 605..."
4,0.851,"[[-15.866860500781998, -47.96679853983349, 605..."
...,...,...
472,0.844,"[[-15.880512000000007, -48.122922, 7626], [-15..."
473,0.845,"[[-15.898521115410318, -48.12156066472963, 411..."
474,0.172,"[[-15.866860500781998, -47.96679853983349, 605..."
475,805.6,"[[-15.86461088791683, -48.02439841365415, 5335..."


In [48]:
print(df.linha[0]) # linha de ida da 0.018
df.parada[0] # Verificando os pontos de paradas com seus respectivos números. 

0.018


[[-15.767293, -47.788723, '7557'],
 [-15.770124999999993, -47.788733, '7558'],
 [-15.76816299999999, -47.79217, '7559'],
 [-15.76878769178167, -47.79181613649742, '6849'],
 [-15.80215197439144, -47.85334236319928, '3274'],
 [-15.810564155657119, -47.84685850621892, '2442'],
 [-15.736023802288491, -47.9043530752539, '6287'],
 [-15.803428845776875, -47.79425505968125, '2396'],
 [-15.809442370814327, -47.80092191122717, '2395'],
 [-15.803454253894726, -47.79461287922536, '2397'],
 [-15.829392241144253, -47.81058449567555, '2389'],
 [-15.831581136451492, -47.8129618471711, '2388'],
 [-15.798766726003029, -47.87085223489018, '3270'],
 [-15.799287300335465, -47.86919901400948, '3271'],
 [-15.798242350178315, -47.872512648437414, '3269'],
 [-15.800349949694871, -47.865823638647186, '3272'],
 [-15.80140481164999, -47.86245692727506, '3273'],
 [-15.817478071398625, -47.805750824839436, '2393'],
 [-15.822977042333191, -47.805016085485335, '2391'],
 [-15.820146118518355, -47.80504912316825, '2392

In [49]:
# Trabalhando com o dataframe criado
# Primeiro convertendo para array m*3
import numpy as np
a = np.array(df.parada[0])
a.shape

(58, 3)

In [50]:
import folium
from shapely.geometry import LineString

df1 = df[df['linha']=='0.813']


linha_exemplo = df1.iloc[0]

linha_coordinates = [(lat, lon) for lat, lon, _ in linha_exemplo['parada']]
paradas_coordinates = [(lat, lon) for lat, lon, _ in linha_exemplo['parada']]

linha_onibus = LineString(linha_coordinates)

m = folium.Map(location=[linha_coordinates[0][0], linha_coordinates[0][1]], zoom_start=13)

for lat, lon, popup_text in linha_exemplo['parada']:
    folium.Marker([lat, lon], popup=popup_text).add_to(m)
m


In [51]:
import os

if not os.path.exists('../databases/luiz_dataset'):
    os.makedirs('../databases/luiz_dataset')
df.to_csv('../databases/luiz_dataset/paradas_linestrings_volta.csv')

### Como ler o arquivo das paradas corretamente no python

Para evitar o erro do tipo de dado das linhas serem strings, é necessário passar para o atributo
'converters' = {'NOME DA COLUNA': pd.eval}
```python
dt = pd.read_csv("../databases/luiz_dataset/paradas_linestrings.csv", converters={'parada': pd.eval})
```